In [8]:
import pd
import torch

if torch.cuda.is_available():
    print("CUDA is available.")
else:
    print("CUDA is not available. Running on CPU-only mode.")

CUDA is available.


In [9]:
import torch
print("CUDA dostępne:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Wykryta karta:", torch.cuda.get_device_name(0))


CUDA dostępne: True
Wykryta karta: NVIDIA GeForce RTX 3060 Laptop GPU


In [11]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.112  Python-3.12.3 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Setup complete  (16 CPUs, 15.7 GB RAM, 165.6/200.1 GB disk)


# test


In [2]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


Looking in indexes: https://download.pytorch.org/whl/cu121
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
print("CUDA dostępne:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Wykryta karta:", torch.cuda.get_device_name(0))

CUDA dostępne: True
Wykryta karta: NVIDIA GeForce RTX 3060 Laptop GPU


In [4]:
import sys
print(sys.executable)

C:\Users\USER\AppData\Local\Programs\Python\Python312\python.exe


In [5]:

%pip install ipykernel
%python -m ipykernel install --user --name cuda-env --display-name "Python (CUDA)"


Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).


In [6]:
import torch
print(torch.__version__)
print("CUDA:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Brak")



2.5.1+cu121
CUDA: True
GPU: NVIDIA GeForce RTX 3060 Laptop GPU


# trening

In [10]:
from ultralytics import YOLO

# Load a last_model
model = YOLO("yolo11n.pt")  # load a pretrained last_model

# Train the last_model on GPU (device=0)
results = model.train(
    data=r"C:\Users\USER\Downloads\dataset\data.yaml",
    epochs=100,
    imgsz=512,
    device=0  # GPU 0 = Twoja karta NVIDIA
)

100%|██████████| 5.35M/5.35M [00:01<00:00, 4.64MB/s]


New https://pypi.org/project/ultralytics/8.3.127 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.112  Python-3.12.3 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=C:\Users\USER\Downloads\dataset\data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=

RuntimeError: Dataset 'C://Users/USER/Downloads/dataset/data.yaml' error  'C:\Users\USER\Downloads\dataset\data.yaml' does not exist

# gotowy model

In [ ]:
from ultralytics import YOLO

# Load a last_model
model = YOLO("model2.0/weights/best.pt")  # ładuje last_model lokalnie z projektu

# Inference using the last_model
results = model.predict("C:/Users/basia/PycharmProjects/proj1-nowotwor/last_model/output3/test/images")

In [ ]:
results = pd.DataFrame(results)

In [ ]:
import os
import pandas as pd

# Ścieżki do folderów
images_folder = "C:/Users/basia/PycharmProjects/proj1-nowotwor/last_model/output3/test/images"
labels_folder = "C:/Users/basia/PycharmProjects/proj1-nowotwor/last_model/output3/test/labels"

# Lista do przechowywania danych
data = []

# Iteracja po plikach graficznych
for image_file in os.listdir(images_folder):
    if image_file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
        image_name = image_file
        label_file = os.path.splitext(image_file)[0] + ".txt"
        label_path = os.path.join(labels_folder, label_file)

        # Sprawdzenie czy odpowiadający plik tekstowy istnieje
        if os.path.isfile(label_path):
            with open(label_path, 'r', encoding='utf-8') as f:
                first_char = f.read(1)
                label = int(first_char) if first_char in ['0', '1'] else None
        else:
            label = None  # lub np. -1 jeśli chcesz oznaczyć brak pliku

        data.append({'image_name': image_name, 'label': label})

# Tworzenie DataFrame
test_real = pd.DataFrame(data)

# Wyświetlenie lub zapisanie wyników
print(df)
# df.to_csv("image_labels.csv", index=False)  # jeśli chcesz zapisać


In [ ]:
import os
import pandas as pd
import numpy as np

data = []

# Sprawdzamy, czy to pojedynczy obiekt Results, czy tablica (np. z predict())
results_array = results.values
results_flat = results_array.flatten() if isinstance(results_array, np.ndarray) else [results_array]

for result in results_flat:
    image_path = result.path
    image_name = os.path.basename(image_path)

    if result.boxes is not None and result.boxes.cls.numel() > 0:
        labels = result.boxes.cls.cpu().numpy().astype(int)
        label = int(pd.Series(labels).mode()[0])  # Najczęstszy label
    else:
        label = None  # Brak wykrycia

    data.append({'image_name': image_name, 'label': label})

# Tworzymy DataFrame
df = pd.DataFrame(data)
print(df)


In [ ]:
test_pred = df
test_pred = test_pred.fillna(0)

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score

# LEFT JOIN po image_name
df = test_real.merge(test_pred, on='image_name')

# Uzupełnienie brakujących predykcji na 0 (brak wykrycia = negatyw)
df['label_pred'] = df['label_pred'].fillna(0).astype(int)

# Obliczenie metryk
accuracy = accuracy_score(df['label_real'], df['label_pred'])
precision = precision_score(df['label_real'], df['label_pred'], zero_division=0)
recall = recall_score(df['label_real'], df['label_pred'], zero_division=0)

# Wyświetlenie wyników
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
